<a href="https://colab.research.google.com/github/robd123/IndepResearch/blob/master/BugClassificationBERT_TypeOnly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
model_name = "t5-small"
token_len = 512
model_prefix = f"{model_name}-{token_len}"

In [2]:
pip install sentencepiece

In [3]:
#pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl


In [4]:
#pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchtext==0.10.0 -f https://download.pytorch.org/whl/cu111/torch_stable.html

In [5]:
pip install pytorch-lightning

In [6]:
pip install nlp

In [7]:
pip install transformers

In [8]:
pip install wandb

In [9]:
!wandb login

wandb: Currently logged in as: robd123 (use `wandb login --relogin` to force relogin)


In [10]:
import torch
from pytorch_lightning.loggers.wandb import WandbLogger
import os
from pathlib import Path
from string import punctuation

os.environ["WANDB_API_KEY"] = '33461c97826e5a650bb71bec8b3e694e7c6ee9f6'
wandb_logger = WandbLogger(project='bugclassification-t5small')

In [11]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import string
import pandas as pd
# pd.set_option('display.max_colwidth', -1)
import numpy as np
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.loggers import WandbLogger
from nlp import load_metric

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup,
    BertForSequenceClassification,
    BertTokenizer,
)
import random
import re


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
from google.colab import auth
auth.authenticate_user()

!gcloud config set project eternal-bruin-340122

!gsutil cp gs://robd_indep_research/linux_bugs_usage_ready.csv /tmp/dataset.csv

Updated property [core/project].
Copying gs://robd_indep_research/linux_bugs_usage_ready.csv...
/ [1 files][ 40.8 MiB/ 40.8 MiB]                                                
Operation completed over 1 objects/40.8 MiB.                                     


Text Processing Functions
From https://github.com/Forethought-Technologies/ieee-dsmp-2018-paper/blob/master/text_processing.py

In [13]:
import string
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

def strip_punctuations(data, column_name='text'):
  '''
  Strips punctuations from the end of each token.
  This uses suggestion from https://stackoverflow.com/questions/34293875/how-to-remove-punctuation-marks-from-a-string-in-python-3-x-using-translate
  to accomplish this really fast.
  '''
  translator = str.maketrans('', '', string.punctuation)
  data['text'] = data['text'].map(lambda s : str(s).translate(translator))
  return data

def stemm_text(data, stemmer_choice='Lancaster'):
  '''
  Stemm the 'text' column of data - this simplifies the words so
  that different forms of the same word end up being the same.
  '''
  if stemmer_choice == 'Lancaster':
    stemmer = LancasterStemmer()
  elif stemmer_choice == 'Snowball':
    stemmer = SnowballStemmer('english')
  elif stemmer_choice == 'Porter':
    stemmer = PorterStemmer()
  else:
    raise Exception('Illegal stemmer_choice argument')
  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: stemmer.stem(w), s.split())))
  return data

def remove_long_words(data, maxlen=16):
  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if len(w) < maxlen else ' ', s.split())))
  return data

def remove_short_words(data, minlen=4):
  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if len(w) >= minlen else ' ', s.split())))
  return data

def remove_linux_garbage(data):
  '''
  Linux data contains lots of garbage, e.g. memory addresses - 0000f800
  '''
  def is_garbage(w):
    return len(w) >= 7 and sum(c.isdigit() for c in w) >= 2

  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if not is_garbage(w) else ' ', s.split())))
  return data

def cast_to_lowercase(data):
  data['text'] = data['text'].map(lambda s : s.lower())
  return data

def remove_stopwords(data):
  stop_words = stopwords.words('english')
  translator = str.maketrans('', '', string.punctuation)
  stop_words = set([w.translate(translator) for w in stop_words]) # Apostrophes were removed already

  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if w not in stop_words else ' ', s.split())))
  return data

def remove_rare_words(data, min_count=3):
  wc = {} # WordCount
  def proc_word(s):
    for w in set(s.split()):
      if w in wc:
        wc[w] += 1
      else:
        wc[w] = 1

  for index, row in data.iterrows():
    proc_word(row['text'])

  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if wc[w] >= min_count else ' ', s.split())))
  return data

In [14]:
data = pd.read_csv('/tmp/dataset.csv', sep='\t')

In [15]:
data.head()

,title,message,importance,product,component
0,oops when using ide-cd with 2.5.45 and cdrecord,Please enter Exact Kernel version:2.5.45 Distr...,P2 normal,IO/Storage,IDE
1,NUMA-Q hangs during TSC initialization on boot.,Exact Kernel version: 2.5.46 Distribution: deb...,P2 normal,Platform Specific/Hardware,i386
2,Enabling shared pagetables causes KDE to wierd...,Exact Kernel version: 2.5.46-mm1 Distribution:...,P2 normal,Memory Management,Other
3,Dcache spirals out of control on 2.5.43-mm2,Exact Kernel version: 2.5.43-mm2 Distribution:...,P2 blocking,IO/Storage,Other
4,64GB highmem BUG(),Exact Kernel version: 2.5.40 Hardware Environm...,P2 normal,Memory Management,Other


In [16]:
  data['text'] = data['title'] + ' ' + data['message']
  data = data.drop(['title'], axis=1)
  data = data.drop(['message'], axis=1)
  data.head()

,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops when using ide-cd with 2.5.45 and cdrecor...
1,P2 normal,Platform Specific/Hardware,i386,NUMA-Q hangs during TSC initialization on boot...
2,P2 normal,Memory Management,Other,Enabling shared pagetables causes KDE to wierd...
3,P2 blocking,IO/Storage,Other,Dcache spirals out of control on 2.5.43-mm2 Ex...
4,P2 normal,Memory Management,Other,64GB highmem BUG() Exact Kernel version: 2.5.4...


In [17]:
data['prio'] = [a[:2] for a in data['importance']]
data['type'] = [a[3:] for a in data['importance']]

In [18]:
data.drop(data[data['type'] == 'blocking'].index, inplace = True)
data.drop(data[data['type'] == 'enhancement'].index, inplace = True)
data.drop(data[data['type'] == 'empty'].index, inplace = True)

In [19]:
data["type_num"] = [0 if a == 'low' else 1 if a == 'normal' else 2 for a in data["type"]]

In [20]:
data.head()

,importance,product,component,text,prio,type,type_num
0,P2 normal,IO/Storage,IDE,oops when using ide-cd with 2.5.45 and cdrecor...,P2,normal,1
1,P2 normal,Platform Specific/Hardware,i386,NUMA-Q hangs during TSC initialization on boot...,P2,normal,1
2,P2 normal,Memory Management,Other,Enabling shared pagetables causes KDE to wierd...,P2,normal,1
4,P2 normal,Memory Management,Other,64GB highmem BUG() Exact Kernel version: 2.5.4...,P2,normal,1
5,P2 normal,Memory Management,Slab Allocator,2.5.41 oops in kmem_cache_create Exact Kernel ...,P2,normal,1


In [21]:
data.type_num.describe()

count    15373.000000
mean         1.112860
std          0.474238
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          2.000000
Name: type_num, dtype: float64

In [22]:
nltk.download('stopwords')
data = strip_punctuations(data)
data = remove_linux_garbage(data)
data = cast_to_lowercase(data)
data = remove_stopwords(data)
data = remove_rare_words(data)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
data.head()

,importance,product,component,text,prio,type,type_num
0,P2 normal,IO/Storage,IDE,oops using idecd 2545 cdrecord please enter ex...,P2,normal,1
1,P2 normal,Platform Specific/Hardware,i386,numaq hangs tsc initialization boot exact kern...,P2,normal,1
2,P2 normal,Memory Management,Other,enabling shared pagetables causes kde wierd ex...,P2,normal,1
4,P2 normal,Memory Management,Other,64gb highmem bug exact kernel version 2540 har...,P2,normal,1
5,P2 normal,Memory Management,Slab Allocator,2541 oops kmemcachecreate exact kernel version...,P2,normal,1


In [24]:
data.shape

(15373, 7)

In [25]:
import numpy as np
import pandas as pd


class BugData(Dataset):
    def __init__(self, df, tokenizer, type_path, num_samples, input_length=4096, output_length=4096, print_text=False):         
        ## Shuffle data set
        df = df.sample(frac=1, random_state=1)
        self.dataset = []
        val_size = int(0.2 * df.shape[0])

        inps = df["text"].values.tolist()
        lbls = df["type_num"].values.tolist()

        for i, inp in enumerate(inps):
            inp = inp.replace(":", "")
            lbl = str(lbls[i])
            self.dataset.append({"inp": inp, "lbl": lbl})
        
        if type_path == "train":
            self.dataset = self.dataset[:len(self.dataset) - val_size - val_size]
        elif type_path == "validation":
            self.dataset = self.dataset[len(self.dataset) - val_size - val_size: len(self.dataset) - val_size]
        elif type_path == "test":
            self.dataset = self.dataset[len(self.dataset) - val_size:]

        if num_samples:
            self.dataset = self.dataset[:num_samples]
        
        self.input_length = input_length
        self.tokenizer = tokenizer
        self.output_length = output_length
        self.print_text = print_text
        
  
    def __len__(self):
        return len(self.dataset)

    def convert_to_features(self, example_batch):
        
        input_ = example_batch['inp']      
        input_ = input_.strip()      
        source = self.tokenizer.batch_encode_plus([input_], max_length=self.input_length, 
                                                     padding='max_length', truncation=True, return_tensors="pt")
        return source
  
    def __getitem__(self, index):
        source = self.convert_to_features(self.dataset[index])
        
        source_ids = source["input_ids"].squeeze()
        src_mask = source["attention_mask"].squeeze()
        tk_type_ids = source["token_type_ids"].squeeze()

        return {"input_ids": source_ids, "attention_mask": src_mask, "label": torch.tensor(int(self.dataset[index]['lbl'])).long(), "token_type_ids": tk_type_ids}



def get_dataset(tokenizer, type_path, num_samples, args, df=data):
      return BugData(df = df, tokenizer=tokenizer, type_path=type_path, num_samples=num_samples, input_length=args.max_input_length, 
                        output_length=args.max_output_length)

In [26]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
dataset = BugData(data, tokenizer, 'validation', None, token_len, 512, True)
len(dataset)

3074

In [27]:
dataset[3]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0

In [28]:
data_sc = dataset[3]
print()
print("Shape of Tokenized Text: ", data_sc['input_ids'].shape)
print()
print("Sanity check - Decode Text: ", tokenizer.decode(data_sc['input_ids']))


Shape of Tokenized Text:  torch.Size([512])

Sanity check - Decode Text:  [CLS] undefined subject undefined submitter references handledby adrian bunk bunkkernelorg mauro carvalho chehab mchehabinfradeadorg patch [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [29]:
from collections import OrderedDict

class BERTBugClassifier(pl.LightningModule):
    def __init__(self, hparams):
        super(BERTBugClassifier, self).__init__()
        self.save_hyperparameters(hparams)        
        self.model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
        self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
        self.output_dir = Path(self.hparams.output_dir)
        self.step_count = 0

    def ign_forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        return self.model(
                input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                labels=labels
            )
      
    def train_dataloader(self):   
        train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", num_samples=-1, args=self.hparams)
        dataloader = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True, num_workers=1)
        t_total = (
            (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
            // self.hparams.gradient_accumulation_steps
            * float(self.hparams.num_train_epochs)
        )
        scheduler = get_linear_schedule_with_warmup(
            self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
        )
        self.lr_scheduler = scheduler

        return dataloader

    def val_dataloader(self):
        validation_dataset = get_dataset(tokenizer=self.tokenizer, type_path="validation", num_samples=-1, args=self.hparams)
        return DataLoader(validation_dataset, batch_size=self.hparams.eval_batch_size, num_workers=1)
    
    def test_dataloader(self):
        test_dataset = get_dataset(tokenizer=self.tokenizer, type_path="test", num_samples=-1, args=self.hparams)
        return DataLoader(test_dataset, batch_size=self.hparams.eval_batch_size, num_workers=1)

    def training_step(self, batch, batch_idx):
        labels = batch["label"]
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        token_type_ids = batch["token_type_ids"]

        out = self.model(
                input_ids,
                token_type_ids=token_type_ids,
                attention_mask=attention_mask,
                labels=labels
                )
        loss = out[0]

        self.log("val/loss", loss, on_step=True, on_epoch=True)
        tqdm_dict = {"train_loss": loss}
        
        self.log("train/loss", loss, on_step=True, on_epoch=True)
        output = OrderedDict({
            "loss": loss,
            "progress_bar": tqdm_dict,
            "log": tqdm_dict
            })

        return output

    def validation_step(self, batch, batch_idx):
        labels = batch["label"]
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        token_type_ids = batch["token_type_ids"]

        out = self.model(
                input_ids,
                token_type_ids=token_type_ids,
                attention_mask=attention_mask,
                labels=labels
                )
        loss = out[0]

        #self.log("val/loss", loss, on_step=True, on_epoch=True)
        output = OrderedDict({
            "val_loss": loss,
            "batch_size": len(labels)
            })
        return output

    def validation_end(self, outputs):
        val_acc = sum([out["correct_count"] for out in outputs]).float() / sum(out["batch_size"] for out in outputs)
        val_loss = sum([out["val_loss"] for out in outputs]) / len(outputs)
        tqdm_dict = {
                "val_loss": val_loss,
                "val_acc": val_acc,
                }
        result = {"progress_bar": tqdm_dict, "log": tqdm_dict, "val_loss": val_loss}
        return result

    def configure_optimizers(self):

        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
        self.opt = optimizer
        return [optimizer]


In [30]:
args_dict = dict(
    output_dir="", # path to save the checkpoints
    model_name_or_path=model_name,
    tokenizer_name_or_path=model_name,
    max_input_length=token_len,
    max_output_length=token_len,
    freeze_encoder=False,
    freeze_embeds=False,
    learning_rate=2e-5,
    weight_decay=0.1,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=1,
    eval_batch_size=1,
    num_train_epochs=10,
    n_gpu=1,
    gradient_accumulation_steps=8,
    val_check_interval = 0.5, 
    n_val=-1,
    n_train=-1,
    n_test=-1,
    early_stop_callback=False,
    fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    max_grad_norm=1.0, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
)


args_dict.update({'output_dir': "./" + model_prefix + "_final", 'num_train_epochs':5,
             'train_batch_size': 8, 'eval_batch_size': 8, 'resume_from_checkpoint' : 't5-small-512t5-small-512_ckpt_epoch_29.ckpt'})
args = argparse.Namespace(**args_dict)



In [31]:
model_name

't5-small'

In [32]:
train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    precision= 16 if args.fp_16 else 32,
    amp_backend='apex',
    #tpu_cores=8,
    gpus=1,
    amp_level=args.opt_level,
    gradient_clip_val=args.max_grad_norm,
    val_check_interval=args.val_check_interval,
    logger=wandb_logger,
    max_epochs=4,
    # progress_bar_refresh_rate=0
)

model = BERTBugClassifier(args)
trainer = pl.Trainer(**train_params)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [33]:
torch.cuda.empty_cache()

In [ ]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
wandb: Currently logged in as: robd123 (use `wandb login --relogin` to force relogin)



  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 109 M 
--------------------------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
437.938   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/optimization/closure.py:36: LightningDeprecationWarning: One of the returned values {'progress_bar', 'log'} has a `grad_fn`. We will detach it automatically but this behaviour will change in v1.6. Please detach it manually: `return {'loss': ..., 'something': something.detach()}`
  f"One of the returned values {set(extra.keys())} has a `grad_fn`. We will detach it automatically"


Validating: 0it [00:00, ?it/s]

In [ ]:
torch.save(model.state_dict(), '/content/sample_data/bc_type_2_bert_model.pt')

NameError: ignored

In [ ]:
!gsutil cp /content/sample_data/bc_type_2_bert_model.pt gs://robd_indep_research/Results

Copying file:///content/sample_data/bc_type_2_t5_model.pt [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

| [1 files][230.9 MiB/230.9 MiB]                                                
Operation completed over 1 objects/230.9 MiB.                                    


In [ ]:
!gsutil cp gs://robd_indep_research/Results/fn_model.pt /content/sample_data/ 

Copying gs://robd_indep_research/Results/fn_model.pt...
/ [1 files][230.9 MiB/230.9 MiB]                                                
Operation completed over 1 objects/230.9 MiB.                                    


In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
df = data.copy()
dataset = BugData(df, tokenizer, 'test', None, token_len, 3, True)
data_len = len(dataset)
print(data_len)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fb87647d510>> (for pre_run_cell):


Exception: ignored

3074
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fb87647d510>> (for post_run_cell):


Exception: ignored

In [ ]:
model = T5BugClassifier(args)
model.load_state_dict(torch.load('/content/sample_data/fn_model.pt'))

<All keys matched successfully>

In [ ]:
loader = DataLoader(dataset, batch_size=128, shuffle=True)
it = iter(loader)

In [ ]:
model.to('cuda')
dec = []
texts = []
targets = []
num = 0;
for batch in it:
  outs = model.model.generate(
            batch["source_ids"].cuda(),
            attention_mask=batch["source_mask"].cuda(),
            use_cache=True,
            decoder_attention_mask=batch['target_mask'].cuda(),
        )
  dec.extend([tokenizer.decode(ids) for ids in outs])
  texts.extend([tokenizer.decode(ids) for ids in batch['source_ids']])
  targets.extend([tokenizer.decode(ids) for ids in batch['target_ids']])
  num = num + 128
  print(num)  #track number of records

Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  to

In [ ]:
import textwrap
from tqdm.auto import tqdm

In [ ]:
targets2 = [a.replace("<pad>", "").replace(" ", "").replace("</s>", "") for a in targets]
dec2 = [a.replace("<pad>", "").replace("</s>", "").replace(" ", "") for a in dec]

In [ ]:
print(targets2[0])
print(dec2[0])

normal
normal


In [ ]:
len(targets2)

3074

In [ ]:
for i in range(4):
    lines = textwrap.wrap("Input Text:\n%s\n" % texts[i], width=100)
    print("\n".join(lines))
    print("\nActual Class: %s" % targets[i])
    print("\nPredicted Class from T5: %s" % dec[i])
    print("=====================================================================\n")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [ ]:
confusion_matrix(targets2, dec2)

array([[   0,    0,    0,    0],
       [   0,  141,   12,  377],
       [   1,   14,   23,  154],
       [   0,  229,  105, 2018]])

In [ ]:
accuracy_score(targets2, dec2)

0.7098243331164606

In [ ]:
print(classification_report(targets2, dec2))

              precision    recall  f1-score   support

       empty       0.00      0.00      0.00         0
        high       0.37      0.27      0.31       530
         low       0.16      0.12      0.14       192
      normal       0.79      0.86      0.82      2352

    accuracy                           0.71      3074
   macro avg       0.33      0.31      0.32      3074
weighted avg       0.68      0.71      0.69      3074



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
